# Version2 of H20 AutoML
### Previous notebook did not parse the Cabin column


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import h2o
from h2o.automl import H2OAutoML

# initialize h2o and load training data
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.1" 2023-10-17; OpenJDK Runtime Environment Homebrew (build 21.0.1); OpenJDK 64-Bit Server VM Homebrew (build 21.0.1, mixed mode, sharing)
  Starting server from /Users/jimhitt/anaconda3/envs/2023_12_26_KaggleTitanic/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/_7/jc101n2d4n79l6kv7vkhr89r0000gn/T/tmpv4m8fsj6
  JVM stdout: /var/folders/_7/jc101n2d4n79l6kv7vkhr89r0000gn/T/tmpv4m8fsj6/h2o_jimhitt_started_from_python.out
  JVM stderr: /var/folders/_7/jc101n2d4n79l6kv7vkhr89r0000gn/T/tmpv4m8fsj6/h2o_jimhitt_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_from_python_jimhitt_j7zznh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.984 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [2]:
# load data
data_df = pd.read_csv("data/train.csv")

# split Cabin into Deck, Number and Side
data_df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = data_df['Cabin'].str.split('/', expand=True)
# cast Cabin_number as int
data_df['Cabin_Number'] = data_df['Cabin_Number'].astype('float64')
# drop Cabin, PassengerId and Name
data_df.drop(['Cabin', 'PassengerId', 'Name'], axis=1, inplace=True)

# convert data_df into h2o frame
data = h2o.H2OFrame(data_df)

# set target variable
data['Transported'] = data['Transported'].asfactor()

# split data into train and test
train, test = data.split_frame(ratios=[0.8], seed=48)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### Run H20 AutoML

Currently using 15 minutes (prior run used 60 minutes)

In [3]:
# Configure and run AutoML
# max_runtime_secs is the time limit; adjust as needed
automl = H2OAutoML(max_runtime_secs=15 * 60, seed=1, project_name='kaggle_project')
automl.train(y='Transported', training_frame=train)




AutoML progress: |
12:13:33.561: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),16/73
# GBM base models (used / total),9/44
# DeepLearning base models (used / total),7/26
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [4]:
# view the AutoML Leaderboard
lb = automl.leaderboard
print(lb)

model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse       mse
StackedEnsemble_AllModels_4_AutoML_1_20231230_121333     0.904045   0.380931  0.915927                0.188834  0.351861  0.123806
StackedEnsemble_AllModels_3_AutoML_1_20231230_121333     0.903895   0.381068  0.916139                0.190128  0.352083  0.123962
StackedEnsemble_AllModels_6_AutoML_1_20231230_121333     0.903892   0.381436  0.916025                0.186959  0.352042  0.123934
StackedEnsemble_BestOfFamily_4_AutoML_1_20231230_121333  0.903838   0.381992  0.915858                0.18724   0.352207  0.12405
StackedEnsemble_BestOfFamily_6_AutoML_1_20231230_121333  0.903675   0.382282  0.915667                0.187672  0.352334  0.124139
StackedEnsemble_Best1000_1_AutoML_1_20231230_121333      0.903511   0.381668  0.915636                0.189262  0.352262  0.124089
StackedEnsemble_AllModels_1_AutoML_1_20231230_121333     0.901664   0.386208  0.9130

In [5]:
# examine the top model
model_id = lb[0, 'model_id']

model = h2o.get_model(model_id)

model.summary()



key,value
Stacking strategy,cross_validation
Number of base models (used / total),16/73
# GBM base models (used / total),9/44
# DeepLearning base models (used / total),7/26
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [6]:
# explore model parameters
model_params = model.params
print(model_params)

# get model performance
model_performance = model.model_performance()
print(model_performance)

{'model_id': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'StackedEnsemble_AllModels_4_AutoML_1_20231230_121333', 'type': 'Key<Model>', 'URL': '/3/Models/StackedEnsemble_AllModels_4_AutoML_1_20231230_121333'}, 'input': None}, 'training_frame': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'AutoML_1_20231230_121333_training_py_3_sid_9f0f', 'type': 'Key<Frame>', 'URL': '/3/Frames/AutoML_1_20231230_121333_training_py_3_sid_9f0f'}, 'input': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'AutoML_1_20231230_121333_training_py_3_sid_9f0f', 'type': 'Key<Frame>', 'URL': '/3/Frames/AutoML_1_20231230_121333_training_py_3_sid_9f0f'}}, 'response_column': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ColSpecifierV3', 'schema_type': 'VecSpecifier'}, 'colum

In [7]:
# run the model on the new test data
test_data_df = pd.read_csv("data/test.csv")

# save the PassengerId for submission
passenger_id = test_data_df['PassengerId']

# parse the Cabin column
test_data_df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = test_data_df['Cabin'].str.split('/', expand=True)
# cast Cabin_number as int
test_data_df['Cabin_Number'] = test_data_df['Cabin_Number'].astype('float64')
# drop Cabin
test_data_df.drop(['Cabin', 'PassengerId', 'Name'], axis=1, inplace=True)

# convert test_data_df into h2o frame
test_data = h2o.H2OFrame(test_data_df)

# get the best model
model_best = automl.leader

# make predictions and convert to df
preds = model_best.predict(test_data)
preds_df = preds.as_data_frame()

# print the predictions
print(preds_df.head())


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
   predict     False      True
0     True  0.490766  0.509234
1    False  0.984360  0.015640
2     True  0.000659  0.999341
3     True  0.014297  0.985703
4     True  0.440730  0.559270


/Users/jimhitt/anaconda3/envs/2023_12_26_KaggleTitanic/lib/python3.11/site-packages/h2o/frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


### Will see how this one does. It was a shorter training period (15 min vs 60 min)


In [8]:
submission = pd.DataFrame({'PassengerId': passenger_id, 'Transported': preds_df['predict']})

# save the submission file as csv
submission.to_csv('data/submission_autoML_02.csv', index=False)

## Splitting the Cabin column improved performance
### Previous best performance 
| Rank | Model | Performance |
| ---- | ----- | ----------- |
| 1 | Deep Neural Net | 80.476% |
| 2 | XGBoost | 80.243% |
| 3 | AutoML_02 | 80.079% |
| 4 | AutoML_01 | 79.401% |



In [11]:
# save the top 3 models
h2o.save_model(model=model_best, path="models/AutoML_02", force=True)
h2o.save_model(model = h2o.get_model(lb[1, 'model_id']), path="models/AutoML_02", force=True)
h2o.save_model(model = h2o.get_model(lb[2, 'model_id']), path="models/AutoML_02", force=True)



'/Users/jimhitt/Code/Kaggle_SpaceshipTitanic/models/AutoML_02/StackedEnsemble_AllModels_6_AutoML_1_20231230_121333'

In [12]:
# shutdown h2o
h2o.cluster().shutdown()

H2O session _sid_9f0f closed.
